In [96]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import pathlib as pl
import os 
import pandas as pd

# Metrics for Model Selection

In this notebook you will fit polynomials to data to decide which order of polynomial is the best fit. Unlike before, the data you will be using is 3 dimensional, meaning it isn't possible to plot. Instead, you will write functions to calculate various metrics that are used to determine model fit. 

Complete this notebook, then answer the questions that go along side it. 

In [97]:
# set random seed for reproducibility
seed = 2022
np.random.seed(seed)

## Load the data 

In [98]:
path_csv = pl.Path(os.getcwd()) / f'M6_Performance_Metrics_Data.csv'
with open(path_csv, 'rb') as file:
    data_pd = pd.read_csv(file)

In [100]:
data_pd.head(10)

,Unnamed: 0,x1,x2,x3,y
0,0,0.382303,-1.596593,1.233776,4.935364
1,1,1.902436,1.579109,-0.341741,25.138660
2,2,-1.689244,1.298489,-1.472081,-4.786340
3,3,-1.510509,1.937616,-1.600244,-3.185759
4,4,1.621717,0.515558,-1.869644,19.712731
5,5,-1.928868,-1.475115,-0.677217,24.007974
6,6,-0.581466,-1.672979,-0.331812,3.786323
7,7,1.409334,-1.663275,-0.899389,-5.856637
8,8,-1.068701,-0.764019,0.805561,5.126408
9,9,-0.473046,-0.477313,-0.483007,3.306021


In [101]:
### Seperating X and Y Variables
data = {'X':data_pd[['x1','x2','x3']].to_numpy(), 'Y':data_pd['y'].to_numpy()}

In [102]:
data

{'X': array([[ 0.38230331, -1.59659343,  1.23377604],
        [ 1.90243561,  1.57910851, -0.34174119],
        [-1.68924391,  1.29848904, -1.47208051],
        [-1.51050932,  1.93761581, -1.60024351],
        [ 1.62171694,  0.51555753, -1.86964382],
        [-1.9288683 , -1.47511467, -0.67721668],
        [-0.58146615, -1.6729788 , -0.33181156],
        [ 1.40933377, -1.66327522, -0.89938906],
        [-1.06870128, -0.76401902,  0.80556103],
        [-0.47304578, -0.47731275, -0.48300685],
        [ 1.00948997, -0.52657911, -1.68216456],
        [-1.79188087,  1.24753051, -1.40480221],
        [-1.09610647,  0.56015469, -0.0722571 ],
        [ 1.62968853,  1.22844166, -1.95196277],
        [ 1.88998474,  0.46476418, -1.29730356],
        [-0.58535968, -0.02372768,  1.0327416 ],
        [-1.14400211,  0.94979955,  0.86627203],
        [-0.88016357,  0.07727138, -1.17613946],
        [-0.43091588,  1.40112791, -0.4806406 ],
        [ 1.06911117, -1.18452612,  0.5760389 ],
        [ 1.148

In [103]:
##index  =   [0    1     2]
data_split = [0.4,0.30,0.30] ## Training, Validation , Testing

## Section 1 : Split the data into training, validation and test sets

### TO DO: write a function that splits the data into traning, validation and test sets.

The function should take as inputs the dataframe and the percentage splits for each of training, validation and test. It should output 3 dataframes, one for each of the sets. 

In [105]:
## write your function here ##
def split_data(data_dict, data_split):
    """divide the data into training, validate and test sets. 
    :param data_dict: a dictionary of the data with keys 'X' and 'Y'
    :param data_split: a list of the fraction of the data to be in each set of form 
    [training_fraction, validation_fraction, test_fraction]. The fractions should all add up to 1.
    :returns training_dict, validation_dict, test_dict: dictionaries of the same form as the data_dict, 
    containing the different sets"""
    
    #assert np.sum(data_split) == 1.0
    
    # work out how many datapoints will be in the train and validation sets 
    n_train = int(len((data_dict['X']))*data_split[0])
    n_validate = int(len((data_dict['X']))*data_split[1])
    
    # generate a random permutation of indices of the data and split into training, validation and test
    perm = np.random.permutation(range(len(data_dict['X'])))
    indices_train, indices_validate, indices_test = np.split(perm, [n_train, n_train+ n_validate])
    
    # create training, validation and test dictionaries 
    training_dict = {'X': data['X'][indices_train], 'Y': data['Y'][indices_train]}
    validation_dict = {'X': data['X'][indices_validate], 'Y': data['Y'][indices_validate]}
    test_dict = {'X': data['X'][indices_test], 'Y': data['Y'][indices_test]}
    
    return training_dict, validation_dict, test_dict

### TO DO: Use your function to split the data so the training set has 40% of the data and the validation and test sets have 30% of the data each

In [106]:
#### write your code here ####
training_data, validation_data , test_data = split_data(data,data_split)

## Section 2: Write Metrics Functions 

### TO DO: Write the functions that calcluate the metrics you will use to evaluate the model fits

Write Functions that return:
- The mean absolute error
- The average error
- The mean absolute percentage error 
- The root mean squared error 
- The total sum of squared errors 

In [107]:
## write your code here ##
import numpy as np

def mean_absolute_error(y_true, y_pred):
    """
    Calculates the mean absolute error between the true and predicted values.

    Parameters:
    y_true (numpy array): An array of true values
    y_pred (numpy array): An array of predicted values

    Returns:
    float: The mean absolute error
    """
    return np.mean(np.abs(y_true - y_pred))


def average_error(y_true, y_pred):
    """
    Calculates the average error between the true and predicted values.

    Parameters:
    y_true (numpy array): An array of true values
    y_pred (numpy array): An array of predicted values

    Returns:
    float: The average error
    """
    return np.mean(y_true - y_pred)


def mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculates the mean absolute percentage error between the true and predicted values.

    Parameters:
    y_true (numpy array): An array of true values
    y_pred (numpy array): An array of predicted values

    Returns:
    float: The mean absolute percentage error
    """
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def root_mean_squared_error(y_true, y_pred):
    """
    Calculates the root mean squared error between the true and predicted values.

    Parameters:
    y_true (numpy array): An array of true values
    y_pred (numpy array): An array of predicted values

    Returns:
    float: The root mean squared error
    """
    return np.sqrt(np.mean((y_true - y_pred)**2))


def total_sum_of_squared_errors(y_true, y_pred):
    """
    Calculates the total sum of squared errors between the true and predicted values.

    Parameters:
    y_true (numpy array): An array of true values
    y_pred (numpy array): An array of predicted values

    Returns:
    float: The total sum of squared errors
    """
    return np.sum((y_true - y_pred)**2)


## Section 3: Fit models to training data and calculate performance metric on validation sets

For polynomials of order 1, 2, 3, and 4, you will use fit_model to fit each each model. This function uses scikit-learn polynomial regression. 


### TODO: write function to convert dataframe into numpy arrays

The scikit-learn functions take numpy arrays as their inputs. Therefore before you can fit any data you need to write a function to turn a dataframe with columns [x1, x2, x3, y] into two numpy arrays: X and y. X should have dimensions (N, D), where N is the number of data points and D is the dimensionality of the data (in this case 3). y should have dimensions (N, ). 


In [108]:
def dataframe_to_numpy(data):
    """
    Converts a dictionary with keys 'X' and 'Y' into NumPy arrays.

    Parameters:
    data (dict): A dictionary with keys 'X' and 'Y'. 'X' should be an array
    of shape (N, D), and 'Y' should be an array of shape (N, ).

    Returns:
    tuple: A tuple containing two NumPy arrays, X and y, where X has shape
    (N, D) and y has shape (N, ).
    """
    X = data['X']
    y = data['Y']
    return X, y


In [109]:
X_train, y_train = dataframe_to_numpy(training_data)
print(X_train.shape)

(40, 3)


In [110]:
X_valid,y_valid = dataframe_to_numpy(validation_data)
print(X_valid.shape)

(30, 3)


In [111]:
X_test,y_test = dataframe_to_numpy(test_data)
print(X_test.shape)

(30, 3)


In [112]:
def fit_model(X, y, order):
    """creates scikit-learn regression object and fits it to the X and y data"""
    model = Pipeline([('poly', PolynomialFeatures(degree=order)),
                      ('linear', LinearRegression(fit_intercept=False))])
    model = model.fit(X, y)
    return model 

### TO DO: For polynomials of order 1 to 6 inclusive: 
1. Fit a polynomial to the training data using the fit_model function 
2. Use model.predict(X) to get the model predictions on the validation set
3. Store the model in a dictionary of models where the keys indicate the order and the items are the models
4. Store the predictions in a seperate dictionary where the keys indicate the order and the items are numpy arrays of the predictions 

In [113]:
# Initialize dictionaries to store models and predictions
models = {}
predictions = {}

# Fit polynomial models of order 1 to 6 to the training data
for order in range(1, 7):
    # Create polynomial features up to the specified order

    # Fit a linear regression model to the polynomial features
    model = fit_model(X_train,y_train,order)

    # Store the model in the dictionary of models
    models[order] = model

    # Get the model predictions on the validation set
    y_val_pred = model.predict(X_valid)

    # Store the predictions in the dictionary of predictions
    predictions[order] = y_val_pred


In [115]:
predictions

{1: array([ 9.48085431,  9.81831768,  3.3889457 ,  3.45848688,  1.61830764,
         4.22070127,  7.39520479, 11.22467846,  2.81988671,  1.68924521,
         4.16323117,  4.24076468, -0.73731226,  2.72506541,  9.49174355,
         6.89409983,  3.82621667,  8.63880161,  6.39116456,  1.3829279 ,
         7.21337138,  5.44499217,  4.80367641,  4.03202368, 10.8754501 ,
        11.07284063,  8.32233985,  6.49794171,  0.33190409,  5.73912101]),
 2: array([19.53825117, -4.47760899,  3.29923085, 11.29503198,  5.34865337,
         7.9798826 , -2.65334196, 13.4371409 ,  4.3702761 , 14.08953562,
        12.20173883,  6.41755756,  5.66442401, 16.11879512,  3.7396162 ,
         6.17540455, 10.32652424,  0.7740557 , 14.55650778,  3.7911862 ,
        12.51508873,  5.91737812,  3.77001139, -7.49144316, 16.82101087,
         3.97869755, 14.21571665,  2.93715453, 20.73914915,  0.2545245 ]),
 3: array([ 21.08002986,  -3.13923969,   4.74714807,  -3.58742656,
          2.56718173,  11.68381646,  -2.0893165

In [116]:
models

{1: Pipeline(steps=[('poly', PolynomialFeatures(degree=1)),
                 ('linear', LinearRegression(fit_intercept=False))]),
 2: Pipeline(steps=[('poly', PolynomialFeatures()),
                 ('linear', LinearRegression(fit_intercept=False))]),
 3: Pipeline(steps=[('poly', PolynomialFeatures(degree=3)),
                 ('linear', LinearRegression(fit_intercept=False))]),
 4: Pipeline(steps=[('poly', PolynomialFeatures(degree=4)),
                 ('linear', LinearRegression(fit_intercept=False))]),
 5: Pipeline(steps=[('poly', PolynomialFeatures(degree=5)),
                 ('linear', LinearRegression(fit_intercept=False))]),
 6: Pipeline(steps=[('poly', PolynomialFeatures(degree=6)),
                 ('linear', LinearRegression(fit_intercept=False))])}

## Section 4: Calculate metrics for each of the models

Now we want to calculate the metrics for each of the models. 


### TODO: Use the dictionary of predictions you have to caluclate and record (could be in a dataframe, or you could plot it on a graph) each of the metrics. 
1. Calculate each of the metrics for the model using the functions you wrote before
2. Store the metrics in a dataframe, with one row for each model or plot on a graph
3. Answer the questions that go alongside this notebook 

HINT: you can write a list of functions of the form:

methods = [RMSE, average_error, mean_abs_percent_error, total_sum_squared_error]

which you can then iterate over using a for loop. 



In [117]:
# Define list of metric functions
methods = [mean_absolute_error, average_error, mean_absolute_percentage_error, root_mean_squared_error, total_sum_of_squared_errors]

# Initialize dataframe to store results
results_df = pd.DataFrame(columns=['order'] + [method.__name__ for method in methods])

# Calculate and record metrics for each model
for order, y_pred in predictions.items():
    row = {'order': order}
    for method in methods:
        metric_value = method(y_valid, y_pred)
        row[method.__name__] = metric_value
    results_df = results_df.append(row, ignore_index=True)

# Print results
print(results_df)


  order mean_absolute_error average_error mean_absolute_percentage_error  \
0   1.0            8.046605      1.985117                     167.191197   
1   2.0            2.621087      0.012278                      64.003596   
2   3.0            5.376502      2.029866                      72.461973   
3   4.0           11.425884      0.860409                     193.892348   
4   5.0            8.032704      3.887179                      87.313374   
5   6.0            7.627031      2.446872                      93.855029   

  root_mean_squared_error total_sum_of_squared_errors  
0                10.31336                  3190.96187  
1                 3.79615                  432.322662  
2                7.807554                  1828.73693  
3               18.961277                10785.900954  
4               13.576144                 5529.350242  
5               11.595363                 4033.573457  


C:\Users\Abdul Basit Aftab\AppData\Local\Temp\ipykernel_9240\1701155533.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Abdul Basit Aftab\AppData\Local\Temp\ipykernel_9240\1701155533.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Abdul Basit Aftab\AppData\Local\Temp\ipykernel_9240\1701155533.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Abdul Basit Aftab\AppData\Local\Temp\ipykernel_9240\1701155533.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pand

## Section 5: Use the test set to evaluate the performance of your chosen model

### TODO: For your selected model, calculate the RMSE, Average Error and Mean Absolute Percentage Error of the test data

In [118]:
test_models = models
test_predictions = {}
for order in range(1, 7):
    # Store the model in the dictionary of models
    test_models[order] = models[order]

    # Get the model predictions on the validation set
    y_test_pred = models[order].predict(X_test)

    # Store the predictions in the dictionary of predictions
    test_predictions[order] = y_test_pred


In [91]:
test_models

{1: Pipeline(steps=[('poly', PolynomialFeatures(degree=1)),
                 ('linear', LinearRegression(fit_intercept=False))]),
 2: Pipeline(steps=[('poly', PolynomialFeatures()),
                 ('linear', LinearRegression(fit_intercept=False))]),
 3: Pipeline(steps=[('poly', PolynomialFeatures(degree=3)),
                 ('linear', LinearRegression(fit_intercept=False))]),
 4: Pipeline(steps=[('poly', PolynomialFeatures(degree=4)),
                 ('linear', LinearRegression(fit_intercept=False))]),
 5: Pipeline(steps=[('poly', PolynomialFeatures(degree=5)),
                 ('linear', LinearRegression(fit_intercept=False))]),
 6: Pipeline(steps=[('poly', PolynomialFeatures(degree=6)),
                 ('linear', LinearRegression(fit_intercept=False))])}

In [92]:
root_mean_squared_error(y_test, new_pred)

8.640523062865842

In [119]:
## write your code here ## 
# Initialize dataframe to store results
test_methods = methods = [average_error, mean_absolute_percentage_error, root_mean_squared_error]
test_results_df = pd.DataFrame(columns=['order'] + [method.__name__ for method in test_methods])

# Calculate and record metrics for each model
for order, y_pred in test_predictions.items():
    row = {'order': order}
    for method in test_methods:
        metric_value = method(y_test, y_pred)
        row[method.__name__] = metric_value
    test_results_df = test_results_df.append(row, ignore_index=True)

# Print results
print(test_results_df)

  order average_error mean_absolute_percentage_error root_mean_squared_error
0   1.0     -1.891249                      300.26961                8.040202
1   2.0     -1.185284                     224.169706                4.157551
2   3.0     -0.616602                     228.019542                5.855951
3   4.0     -6.388542                     491.161582               23.449893
4   5.0     -1.949884                     198.802914               10.322205
5   6.0     -2.657774                     201.665538               12.608421


C:\Users\Abdul Basit Aftab\AppData\Local\Temp\ipykernel_9240\1013619281.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_results_df = test_results_df.append(row, ignore_index=True)
C:\Users\Abdul Basit Aftab\AppData\Local\Temp\ipykernel_9240\1013619281.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_results_df = test_results_df.append(row, ignore_index=True)
C:\Users\Abdul Basit Aftab\AppData\Local\Temp\ipykernel_9240\1013619281.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_results_df = test_results_df.append(row, ignore_index=True)
C:\Users\Abdul Basit Aftab\AppData\Local\Temp\ipykernel_9240\1013619281.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas